In [1]:
import os

os.environ['create_image']='True'
os.environ['repository']='docker.io/romeokienzler'
os.environ['version']='0.1'
os.environ['name']='claimed-sim-wrf'

In [3]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM romeokienzler/mywrfcontainerimage
    WORKDIR /wrf/WRF
    RUN ./clean -a
    RUN ./configure
    RUN ./compile em_real >&! foo
    WORKDIR /wrf/WPS
    RUN ./configure
    RUN ./compile >&! foo
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t `echo $name`:`echo $version` .
    #!docker tag `echo $name`:`echo $version` `echo $repository`/`echo $name`:`echo $version`
    #!docker push `echo $repository`/`echo $name`:`echo $version`
    !rm Dockerfile
else: 
    None

Sending build context to Docker daemon  14.34kB
Step 1/8 : FROM romeokienzler/mywrfcontainerimage
 ---> 7cf57e8f908b
Step 2/8 : WORKDIR /wrf/WRF
 ---> Running in e689707b92c6
Removing intermediate container e689707b92c6
 ---> 96d7a5ddcc69
Step 3/8 : RUN ./clean -a
 ---> Running in dd4680c29d05
frame
chem
share
dyn_em
dyn_exp
dyn_nmm
phys
main
tools
wrftladj
( cd build; /bin/rm -f *.o *.mod *.f *.c *.m4 *.stb *.out core *~ mpif.h da_generic_boilerplate.inc )
( cd gen_be; /bin/rm -f *.o *.f core *~ )
/bin/rm -f *.o deftab nocontf90 callgraph subinfo_calls
set arg=-a
if ( -a == -a || -a == -aa ) then
if ( -d var ) then
cd var
make superclean
( cd build; /bin/rm -f mpif.h; /bin/rm -f *.inc *.prm; /bin/rm -f *.f90 *.c *.m4 *.exe; \
          /bin/rm -rf *.o *.mod *.b *.a *.f frame/*.F diffwrf *.f90.tmp *.f.tmp *.bb \
          Registry_tmp* module_state_description.f90 \
          inc/dm_comm_cpp_flags testall.csh *.exe *.f90 *.F90 *.F inc/*.inc *.int *.c *.h *.m4 *.code \
          *.sed i

In [ ]:
#! /bin/sh
ls
docker   run   -it   --name   teachme   7cf57e8f908b   /bin/tcsh
ls -ls
cd WRF
./clean -a
./configure
./compile em_real >&! foo
ls -ls main/*.exe
cd ../WPS
./configure
./compile >&! foo
ls -ls *.exe
cp namelist.wps namelist.wps.original
cp /wrf/wrfinput/namelist.wps.docker namelist.wps
./geogrid.exe
ls -ls geo_em.d01.nc
./link_grib.csh /wrf/wrfinput/fnl
cp ungrib/Variable_Tables/Vtable.GFS Vtable
./ungrib.exe
ls -ls FILE*
./metgrid.exe
ls -ls met_em.*
cd ../WRF/test/em_real
ln -sf ../../../WPS/met_em* .
cp namelist.input namelist.input.original
cp /wrf/wrfinput/namelist.input.docker namelist.input
mpirun -np 2 ./real.exe
tail rsl.out.0000
ls -ls wrfinput_d01 wrfbdy_d01
mpirun -np 3 ./wrf.exe &
tail rsl.out.0000
ls -ls wrfo*